In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
basics_df = pd.read_csv('./test-data/title.basics.tsv', sep='\t', na_values='\\N', usecols=['tconst', 'genres', 'startYear'])
basics_df.head()
# ratings_df = pd.read_csv('title.ratings.tsv.gz', sep='\t', na_values='\\N', usecols=['tconst', 'averageRating'])startYear


,tconst,startYear,genres
0,tt0000001,1894.0,"Documentary,Short"
1,tt0000002,1892.0,"Animation,Short"
2,tt0000003,1892.0,"Animation,Comedy,Romance"
3,tt0000004,1892.0,"Animation,Short"
4,tt0000005,1893.0,"Comedy,Short"


In [10]:
ratings_df = pd.read_csv('./test-data/title.ratings.tsv', sep='\t', na_values='\\N', usecols=['tconst', 'averageRating'])
ratings_df.head()

,tconst,averageRating
0,tt0000001,5.7
1,tt0000002,5.8
2,tt0000003,6.5
3,tt0000004,5.6
4,tt0000005,6.2


In [12]:
data_df = pd.merge(basics_df, ratings_df, on='tconst')
data_df.dropna(inplace=True)
data_df.head()

,tconst,startYear,genres,averageRating
0,tt0000001,1894.0,"Documentary,Short",5.7
1,tt0000002,1892.0,"Animation,Short",5.8
2,tt0000003,1892.0,"Animation,Comedy,Romance",6.5
3,tt0000004,1892.0,"Animation,Short",5.6
4,tt0000005,1893.0,"Comedy,Short",6.2


In [13]:
genres = set()
for row in data_df.itertuples():
    for genre in row.genres.split(','):
        genres.add(genre)
genres = sorted(genres)

# Convert the genres into one-hot encoded features
for genre in genres:
    data_df[genre] = data_df.genres.apply(lambda x: 1 if genre in x else 0)


In [14]:
X = data_df.drop(['tconst', 'genres'], axis=1)
y = data_df.genres
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [16]:
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9495219303718493


In [18]:
ratings = 7
movie_df = pd.DataFrame({'averageRating': [ratings]})
for genre in genres:
    movie_df[genre] = 0
y_pred = knn.predict(movie_df)
print(f'The predicted trending genre of the movie is: {y_pred[0]}')

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- startYear
